# Mapping 3
- Line of Interest (circle & straight line)
- Frequency spectrum mapping
- Linear interpolation

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime

import pya
from pya import *

%matplotlib widget

In [ ]:
s = startup()

# Simulation

In [ ]:
import simulation

n = 128
sim_speed = 0.004
sim_fps = 400
duration = 6
frame_amount = duration * sim_fps

initial_state = np.array([[simulation.gaussian(x, y, n, offset=[-0.6, 0.0], width=0.15, height=0.15) for x in range(n)] for y in range(n)])
potential = np.array([[simulation.parabola(x, y, n, offset=(0, 0), factor=(10000, 10000)) for x in range(n)] for y in range(n)])

barrier_x = n//2 - 1
barrier_width = 2
multi_slit = [(-15, -13), (-8, -6), (-1, 1), (6, 8), (13, 15)]
double_slit = [(-4, -2), (2, 4)]
single_slit = [(-2, 2)]
slits = double_slit

frames = simulation.sim(n, sim_fps, duration, slits, barrier_x, barrier_width, sim_speed, initial_state=initial_state, potential=potential, normalize=True)

In [ ]:
import matplotlib
plt.figure()
plt.pcolormesh(np.abs(frames[0]) ** 2, cmap='inferno', norm=matplotlib.colors.PowerNorm(vmin=0, vmax=np.max(np.square(np.abs(frames))), gamma=0.4))
# plt.pcolormesh(potential, vmin=0, vmax=20000)
plt.colorbar()
plt.show()

# Video

In [ ]:
import video

# save video
video_filename, anim = video.create(frames, 20, 1, frame_amount, sim_fps, slits, barrier_x, barrier_width, n)

#plt.close()

# Sonification

In [ ]:
# circle of interest
def circle(radius, center, num_points):
    radians = np.linspace(0, 2 * np.pi, num_points, endpoint=False)
    x = (0.5 * n * (radius * np.sin(radians) + 1 + center[0]))
    y = (0.5 * n * (radius * np.cos(radians) + 1 + center[1]))
    return x, y

# half circle (bottom to top)
def half_circle(radius, center, num_points):
    radians = np.linspace(0, np.pi, num_points, endpoint=True)
    x = (0.5 * n * (radius * np.sin(radians) + 1 + center[0]))
    y = (0.5 * n * (radius * np.cos(radians) + 1 + center[1]))
    return x, y

# line of interest
def line(start, end, num_points):
    t = np.linspace(0, 1, num_points, endpoint=True)
    x = start[0]+n//2 + t * (end[0] - start[0])
    y = start[1]+n//2 + t * (end[1] - start[1])
    return x, y

In [ ]:
np.array(half_circle(0.5, (0,0), 10))

In [ ]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '20'
os.environ['NUMEXPR_NUM_THREADS'] = '16'
import numexpr

def interpolate_parallel(frames, indices, x_vals, y_vals):
    # Hardcoded interpolation
    # Way faster than any other interpolation!
    t = np.array([indices, y_vals, x_vals]) % 1
    t0, t1, t2 = t[0], t[1], t[2]
    floors = np.floor([indices, y_vals, x_vals]).astype(int)
    ceils  = np.ceil ([indices, y_vals, x_vals]).astype(int)
    
    f0 = frames[floors[0], floors[1], floors[2]]
    f1 = frames[floors[0], floors[1], ceils [2]]
    f2 = frames[floors[0], ceils [1], floors[2]]
    f3 = frames[floors[0], ceils [1], ceils [2]]
    f4 = frames[ceils [0], floors[1], floors[2]]
    f5 = frames[ceils [0], floors[1], ceils [2]]
    f6 = frames[ceils [0], ceils [1], floors[2]]
    f7 = frames[ceils [0], ceils [1], ceils [2]]
    
    return numexpr.evaluate('''\\
            ((1-t0) * ((1-t1) * ((1-t2) * real(abs(f0)) ** 2 \\
                              +     t2  * real(abs(f1)) ** 2) \\
                    +     t1  * ((1-t2) * real(abs(f2)) ** 2 \\
                              +     t2  * real(abs(f3)) ** 2)) \\
              + t0  * ((1-t1) * ((1-t2) * real(abs(f4)) ** 2 \\
                              +     t2  * real(abs(f5)) ** 2) \\
                    +     t1  * ((1-t2) * real(abs(f6)) ** 2 \\
                              +     t2  * real(abs(f7)) ** 2))) \\
            ''', local_dict=locals())


def interpolate(frames, indices, x_vals, y_vals):
    t = np.array([indices, y_vals, x_vals]) % 1
    floors = np.floor([indices, y_vals, x_vals]).astype(int)
    ceils  = np.ceil ([indices, y_vals, x_vals]).astype(int)
    
    return ((1-t[0]) * ((1-t[1]) * ((1-t[2]) * np.square(np.abs(frames[floors[0], floors[1], floors[2]]))
                                 +     t[2]  * np.square(np.abs(frames[floors[0], floors[1], ceils [2]])))
                     +     t[1]  * ((1-t[2]) * np.square(np.abs(frames[floors[0], ceils [1], floors[2]]))
                                 +     t[2]  * np.square(np.abs(frames[floors[0], ceils [1], ceils [2]]))))
             + t[0]  * ((1-t[1]) * ((1-t[2]) * np.square(np.abs(frames[ceils [0], floors[1], floors[2]]))
                                 +     t[2]  * np.square(np.abs(frames[ceils [0], floors[1], ceils [2]])))
                     +     t[1]  * ((1-t[2]) * np.square(np.abs(frames[ceils [0], ceils [1], floors[2]]))
                                 +     t[2]  * np.square(np.abs(frames[ceils [0], ceils [1], ceils [2]])))))

In [ ]:
# parameters
sample_rate = 44100
num_frequencies = 40
sample_amount = sample_rate * duration

f_min = 0
f_max = 4000
# linear distribution
frequencies = np.linspace(f_min, f_max, num_frequencies, endpoint=False)
# linear distribution
# frequencies = np.fft.fftfreq(num_frequencies * 2, 1 / sample_rate)[:num_frequencies] / sample_rate*2 * (f_max - f_min) + f_min
# exponential distribution
# frequencies = f_min * (f_max / f_min) ** np.linspace(0, 1, num_frequencies, endpoint=False)

# choose method
do_left = False
# x, y = circle(radius=0.6, center=[0, 0], num_points=num_frequencies)
# x, y = half_circle(radius=0.6, center=[0, 0], num_points=num_frequencies)
x, y = line(start=[-60, 0], end=[60, 0], num_points=num_frequencies)

In [ ]:
# Sonification
frames_indices = np.linspace(0, frame_amount - 1, sample_amount, endpoint=False)

t = np.arange(sample_amount) / sample_rate
sini = np.sin(2 * np.pi * frequencies[np.newaxis, :] * t[:, np.newaxis])
# sini = np.sin(2 * np.pi * ( frequencies[np.newaxis, :] * t[:, np.newaxis] + np.random.random(num_frequencies)[np.newaxis, :]))

f1 = frames_indices[:, np.newaxis].repeat(num_frequencies, axis=1).flatten()
x1 = x[np.newaxis, :].repeat(sample_amount, axis=0).flatten()
y1 = y[np.newaxis, :].repeat(sample_amount, axis=0).flatten()

if do_left:
    x1_left = np.array(n) - x1

In [ ]:
plt.figure()
plt.plot(frequencies)

In [ ]:
# action
audio = interpolate(frames, f1, x1, y1)
audio = np.sum(sini * audio.reshape(sample_amount, num_frequencies), axis=1)

# TODO as=Astft(); as.rfftsig=our_matrix

In [ ]:
if do_left: 
    audio_left = interpolate(frames, f1, x1_left, y1)
    audio_left = np.sum(sini * audio_left.reshape(sample_amount, num_frequencies), axis=1)

In [ ]:
# create pya
pa = Asig(audio, sr=sample_rate).fade_in(0.005).fade_out(0.005).norm().stereo()
save_audio = pa

In [ ]:
# create pya (2 channels)
if do_left:
    pb = Asig(np.array([audio_left, audio]).transpose((1, 0)), sr=sample_rate).fade_in(0.005).fade_out(0.005).norm()
    save_audio = pb

In [ ]:
# save WAV
audio_filename = f'output/sonification_{datetime.now().strftime("%Y_%m_%d-%H_%M_%S")}.wav'
save_audio.save_wavfile(audio_filename)
print(f"Sonification saved as {audio_filename}")

In [ ]:
# play directly
save_audio.play(onset=1, rate=1)

In [ ]:
if do_left:
    plt.figure()
    plt.plot(audio_left)

In [ ]:
plt.figure()
plt.plot(audio)

# Combine Video & Audio

In [ ]:
video.combine(audio_filename, video_filename)